# ESML - accelerator: "Connect to BOTH DEV and TEST workspaces


In [1]:
import sys, os
from azureml.core import Workspace
sys.path.append(os.path.abspath("../azure-enterprise-scale-ml/esml/common/"))  # NOQA: E402
from esml import ESMLDataset, ESMLProject

p = ESMLProject() # Will search in ROOT for your copied SETTINGS folder '../../../settings', you should copy template settings from '../settings'
p.active_model = 11
p.ws = p.get_workspace_from_config() #2) Load DEV or TEST or PROD Azure ML Studio workspace
p.inference_mode = False

In [2]:
print(p.ws.name)
ws2 = p.get_other_workspace("test") # Fetch ws2 via SP, from default keyvault in ws1
print(ws2.name)

msft-weu-DEV-eap-proj02_ai-amls
msft-weu-TEST-eap-proj02_ai-amls


In [3]:
p.dev_test_prod = "dev"
p.describe()

Inference version: 1

 - ds01_diabetes
projects/project002/03_diabetes_model_reg/inference/1/ds01_diabetes/in/dev/2021/06/08/
projects/project002/03_diabetes_model_reg/inference/1/ds01_diabetes/out/bronze/dev/
projects/project002/03_diabetes_model_reg/inference/1/ds01_diabetes/out/silver/dev/

 - ds02_other
projects/project002/03_diabetes_model_reg/inference/1/ds02_other/in/dev/2021/06/08/
projects/project002/03_diabetes_model_reg/inference/1/ds02_other/out/bronze/dev/
projects/project002/03_diabetes_model_reg/inference/1/ds02_other/out/silver/dev/
 

Training GOLD (p.GoldPath)
projects/project002/03_diabetes_model_reg/inference/1/gold/dev/
 

[A) USAGE]: to_score_folder, scored_folder, date_folder = p.get_gold_scored_unique_path()
A)INFERENCE ONLINE: GOLD to score (example if realtime - today)
projects/project002/03_diabetes_model_reg/inference/1/gold/dev/2021_06_21/899e0ca7175f44baad44a099ddd902c3/
 

A)INFERENCE ONLINE: GOLD scored (example if realtime today)
projects/project002/03_

In [4]:
datastore1 = p.init(p.ws) # DEV WS (main workspace - always start at DEV...to get TEST, or PROD)

...
Using GEN2 as Datastore
Searching for setting in ESML datalake...
ESML in-folder settings override = TRUE 
 - Found settings in the ESML AutoLake  [active_in_folder.json,active_scoring_in_folder.json], to override ArgParse/GIT config with.
 - TRAIN in date:  2021/01/01
 - INFERENCE in date: 2021/06/08 and ModelVersion to score with: 1 (0=latest)

Inference mode (False = Training mode): True
Load data as Datasets....
ds01_diabetes
ds02_other

####### Automap & Autoregister - SUCCESS!
1) Auto mapped 2 ESML Dataset with registered Azure ML Datasets (potentially all 3: IN,BRONZE, SILVER) in Datastore project002 

Dataset 'ds01_diabetes' status:
 - IN_Folder_has_files
 - BRONZE_Folder_has_files
 - SILVER_Folder_has_files
Dataset 'ds02_other' status:
 - IN_Folder_has_files
 - BRONZE_Folder_has_files
 - SILVER_Folder_has_files

2) Registered each Dataset with suffixes (_IN, _BRONZE, _SILVER) 
 Tip: Use ESMLProject.Datasets list or .DatasetByName(myDatasetName) to read/write
#######


In [5]:
datastore2 = p.init(ws2) # TEST workspace

...
Using GEN2 as Datastore
Searching for setting in ESML datalake...
ESML in-folder settings override = TRUE 
 - Found settings in the ESML AutoLake  [active_in_folder.json,active_scoring_in_folder.json], to override ArgParse/GIT config with.
 - TRAIN in date:  2021/01/01
 - INFERENCE in date: 2021/06/08 and ModelVersion to score with: 1 (0=latest)

Inference mode (False = Training mode): True
Load data as Datasets....
ds01_diabetes
ds02_other

####### Automap & Autoregister - SUCCESS!
1) Auto mapped 2 ESML Dataset with registered Azure ML Datasets (potentially all 3: IN,BRONZE, SILVER) in Datastore project002 

Dataset 'ds01_diabetes' status:
 - IN_Folder_has_files
 - BRONZE_Folder_has_files
 - SILVER_Folder_has_files
Dataset 'ds02_other' status:
 - IN_Folder_has_files
 - BRONZE_Folder_has_files
 - SILVER_Folder_has_files

2) Registered each Dataset with suffixes (_IN, _BRONZE, _SILVER) 
 Tip: Use ESMLProject.Datasets list or .DatasetByName(myDatasetName) to read/write
#######


from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication

auth = InteractiveLoginAuthentication(tenant_id = p.tenant)
ws = Workspace.get(name = p.workspace_name,subscription_id = p.subscription_id,resource_group = p.resource_group,auth=auth)
ws.write_config(path=".", file_name="../../ws_config.json")

ws = Workspace.from_config("../ws_config.json") # Reads config.json 

# TRAIN MODEL in `TEST` -  First `SAVE a GOLD`

In [15]:
import pandas as pd 
df_01 = p.DatasetByName("ds01_diabetes").Silver.to_pandas_dataframe()
ds_gold_v1 = p.save_gold(df_01)
ds_gold_v1.to_pandas_dataframe().head()

Uploading an estimated of 1 files
Uploading ./common/temp_data/project002/Gold\gold.parquet
Uploaded ./common/temp_data/project002/Gold\gold.parquet, 1 files out of an estimated total of 1
Uploaded 1 files


,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y
0,0.038076,0.05068,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151.0
1,0.085299,0.05068,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141.0
2,0.063504,0.05068,-0.001895,0.066630,0.090620,0.108914,0.022869,0.017703,-0.035817,0.003064,63.0
3,0.041708,0.05068,0.061696,-0.040099,-0.013953,0.006202,-0.028674,-0.002592,-0.014956,0.011349,110.0
4,0.027178,0.05068,0.017506,-0.033214,-0.007073,0.045972,-0.065491,0.071210,-0.096433,-0.059067,69.0


In [16]:
p.dev_test_prod = "test" # to get correct config, compute, etc

" Train in WS2 - TEST Workspace

In [19]:
from azureml.train.automl import AutoMLConfig
from baselayer_azure_ml import AutoMLFactory, ComputeFactory

automl_performance_config = p.get_automl_performance_config()
aml_compute = p.get_training_aml_compute(ws2)

label = p.active_model["label"]
# Returns PANDAS DF's (but also sets p.GoldTrain as Azure Dataset)
train_6, validate_set_2, test_set_2 = p.split_gold_3(0.6,label) # Saves as M03_GOLD_TRAIN | M03_GOLD_VALIDATE | M03_GOLD_TEST # Alt: train,testv= p.Gold.random_split(percentage=0.8, seed=23)

automl_config = AutoMLConfig(task = 'regression',
                             compute_target = aml_compute,
                             training_data = p.GoldTrain, # Azure Dataset
                             label_column_name = label,
                             **automl_performance_config
                            )

best_run, fitted_model, exp = AutoMLFactory(p).train(ws2,automl_config, p.model_folder_name, p.dev_test_prod)


Found existing cluster for project and environment, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
Uploading an estimated of 1 files
Uploading ./common/temp_data/project002/Gold/Train\gold_train.parquet
Uploaded ./common/temp_data/project002/Gold/Train\gold_train.parquet, 1 files out of an estimated total of 1
Uploaded 1 files
Uploading an estimated of 1 files
Uploading ./common/temp_data/project002/Gold/Validate\gold_validate.parquet
Uploaded ./common/temp_data/project002/Gold/Validate\gold_validate.parquet, 1 files out of an estimated total of 1
Uploaded 1 files
Uploading an estimated of 1 files
Uploading ./common/temp_data/project002/Gold/Test\gold_test.parquet
Uploaded ./common/temp_data/project002/Gold/Test\gold_test.parquet, 1 files out of an estimated total of 1
Uploaded 1 files
Experiment name: 03_diabetes_model_reg
Azure ML Studio Workspace: Workspace.create(name='msft-weu-TEST-eap-proj02_ai-amls', subscripti

## 4a) ESML Scoring compare: Promote model or not: `DEV -> TEST`?

In [ ]:
p.dev_test_prod

In [26]:
from baselayer_azure_ml import AutoMLFactory, ComputeFactory
p.dev_test_prod = "test"
print("what environment are we targeting? =  {}".format(p.dev_test_prod)) 
automl_performance_config = p.get_automl_performance_config()
automl_performance_config

what environment are we targeting? =  prod


In [ ]:
from baselayer_azure_ml import AutoMLFactory
p.dev_test_prod = "dev" # Current env, new unregistered model A to validate
target_env = "test" # Target env. Existing registered model B - Does Model A score better than Model B?

print("SCORING DRIFT: If new model scores better in DEV (new data, or new code), we can promote this to TEST & PROD \n")
print(" - If new model scores better, we can register this in DEV/TEST/PROD")
print(" - If new model we trained was DEV workspace, we can register it DEV - or in TEST subscription/workpace.")

promote, m1_name, r1_id, m2_name, r2_run_id = AutoMLFactory(p).compare_scoring_current_vs_new_model(target_env)

print("New Model 1: {}".format(m1_name))
print("Existing Model: {} in environment {}".format(m2_name,target_env))

if (promote and p.dev_test_prod == target_env): # Can only register a model in same ACTIVE workspace (test->test) - need to retrain if going from dev->test
    AutoMLFactory(p).register_active_model(target_env)


# Register model "1 cell code" - no other dependency

In [1]:
sys.path.append(os.path.abspath("../common/"))  # NOQA: E402
from esml import ESMLDataset, ESMLProject
from baselayer_azure_ml import AutoMLFactory
from azureml.core import Workspace

ws = p.get_workspace_from_config()

p = ESMLProject() # Makes it "environment aware (dev,test,prod)", and "configuration aware"
p.dev_test_prod = "test"
p.init(ws) 
AutoMLFactory().register_active_model(p, "test")

Cleaning local temp for project
Using GEN2 as Datastore
ds01_diabetes
Method upload_directory: This is an experimental method, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.
Uploading to datastore: <azureml.data.azure_data_lake_datastore.AzureDataLakeGen2Datastore object at 0x000001C6D50F3CC0> 
From: ./common/temp_data/project002/ds01_diabetes/Bronze/ 
To: master/1_projects/project002/03_diabetes_model_reg/train/ds01_diabetes/out/bronze/ 
Validating arguments.
Arguments validated.
Uploading file to master/1_projects/project002/03_diabetes_model_reg/train/ds01_diabetes/out/bronze/
Uploading an estimated of 1 files
Uploading ./common/temp_data/project002/ds01_diabetes/Bronze\bronze.parquet
Uploaded ./common/temp_data/project002/ds01_diabetes/Bronze\bronze.parquet, 1 files out of an estimated total of 1
Uploaded 1 files
Creating new dataset
ds02_other
Method upload_directory: This is an experimental method, and may change at any time.<br/>For

In [2]:
AutoMLFactory().register_active_model(p, "test")

model.version 3
Model name AutoML7094594e839 is registered.


# ESML - accelerator

## PROJECT + DATA CONCEPTS + ENTERPRISE Datalake Design + DEV->PROD MLOps
- `1)ESML Project`: The ONLY thing you need to remember is your `Project number` (and `BRONZE, SILVER, GOLD` concept )
    - ProjectNo=4 have a list of all your datasets as ESMLDatasets. (Well you need to provide names for them also: "mydata01", "mydata02" - but thats it)
- `2)Lakedesign & Roles`: Bronze, silver, gold + IN and date folders
    - Benefits: Physical datalake design!  onnected to Azure ML Workspace, with autoregistration of `Azure ML Datasets`
    - `Role 1`: `Data ingestion team` only need to care about 1 thing - onboard data to `IN-folder`, in .CSV format
        - `Auto parquet-conversion` from `IN` folder (.CSV) to `OUT`/BRONZE/bronze.PARQUET 
    - `Role 2`: `Data scientists` only need to care about 3 things (R/W): `BRONZE, SILVER, GOLD` datasets, all in .PARQUET format
    - How? The ESML project will `Automap` and `Autoregister` Azure ML Datasets - `IN, SILVER, BRONZE, GOLD`
- `2a) R&D  VS Production phase`: "Latest data" VS versioning on Datasets and datefolders  
    - Benefits "R&D mode": Faster RnD phase to onboard and refresh data easy. Also fast "flip-switch" to production
    - How? `ESMLDataset is context self aware` - knows when it is used in TRAIN or INFERENCE pipeline
- `2b) TRAIN vs INFERENCE` versions</u> `Reuse (Bronze->Silver->Gold) pipepline`, for both TRAIN preprocessing, and INFERENCE 
    - Benefits: Inference with different MODEL version, on data from the same day/time, (to compare scoring etc)
    - How? ESMLDataset have context self awareness, and `knows WHERE and HOW to load/save data`
- `2c) ENTERPRISE CONFIG & PROJECT specicic`: Ajdust, per Dev,Test,Prod settings on ALL projects PERFORMANCE in Dev.
    - Accelerate setup: `Get compute`, `Netwporkijng and private link already set`, `Datadrift, Time series traits, Smart noise, etc`
    - `ShareBack SILVER DATA` from PROJECT to MASTER folder-lake structure: Share refined data back to its "origin/non-projectbased structure" easy: 
        - ESMLProject.ShareBack(ds.Silver)
    - How? ESMProject controls all ESMDatasets, in a uniform way
## ENTERPRISE Deployment of Models & Governance - MLOps  at scale
- `3) DEV->TEST-PROD` (configs, compute, performance)
    - ESML has config for 3 environemnts: Easy DEPLOY model across subscriptions and Azure ML Studio workspaces 
        - Save costs & time: 
            - `DEV` has cheaper compute performance for TRAIN and INFERENCE (batch, AKS)
            - `DEV` has Quick-debug ML training (fast training...VS good scoring in TEST and PROD)
        - How? ESML `AutoMLFactory` and `ComputeFactory`
         

### Q&A:
- Q: Is ESML Machine learning specific? If I only want to refine some data...for integration, or report? 
- A: You can use this for just data refinement also: `Bronze->Silver->Gold` refinement.
    - Benefits: Enterprise security, Read/write to datalake, easy to share refined data. 
    - Benefits: The tooling "glued togehter": Azure datafactory +  Azure Databricks (and Azure ML Studio pipelines if needed)

